In [1]:
import sys
import os 

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

In [2]:
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [ ]:
train.head()

- id : 샘플 아이디
- Gender : 전복 성별
- Lenght : 전복 길이
- Diameter : 전복 둘레
- Height : 전복 키
- Whole Weight : 전복 전체 무게
- Shucked Weight : 껍질을 제외한 무게
- Viscra Weight : 내장 무게
- Shell Weight : 껍질 무게
- Target : 전복 나이

In [ ]:
train.shape

In [4]:
# 결측치 확인
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print (f'결측치가 있는 컬럼은: {col}입니다')
            print (f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다')
            missing_col.append([col, dataframe[col].dtype])
        if counted_missing_col == 0:
            print('결측치가 존재하지 않습니다')
        return missing_col

In [5]:
for i, col in enumerate(train.columns):
    print (i, col)

0 id
1 Gender
2 Lenght
3 Diameter
4 Height
5 Whole Weight
6 Shucked Weight
7 Viscra Weight
8 Shell Weight
9 Target


In [6]:
sum(train['id'].isna())

0

In [7]:
missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


In [8]:
train.isnull().sum()

id                0
Gender            0
Lenght            0
Diameter          0
Height            0
Whole Weight      0
Shucked Weight    0
Viscra Weight     0
Shell Weight      0
Target            0
dtype: int64

In [9]:
# id 제거
train = train.drop(columns=['id'],axis=1)

In [10]:
# target 분포 확인
temp = train['Target'].unique()
np.sort(temp) # 오름차순 정렬

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 29])

In [ ]:
train.groupby('Target').count().iloc[:,0]

In [ ]:
fig, ax = plt.subplots(1, figsize=(25,10))
sns.countplot('Target', data=train)
plt.title("Abalone age by count", fontsize=30)
plt.xlabel("target(age)", fontsize=15)
plt.ylabel("count", fontsize=15)
save_fig("Abalone age by count")
plt.show();

In [ ]:
# 데이터 기초 통계 분석
# 카테고리형 데이터
# 'Gender' 시각화
train['Gender'].unique()

In [ ]:
train.groupby('Gender').count().iloc[:,0]

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot('Gender', data=train)
plt.title("Abalone gender by count", fontsize=30)
save_fig("Abalone gender by count")
plt.show();

In [ ]:
# 성별별로 나이의 분포
plt.figure(figsize=(25, 7))
sns.kdeplot('Target', hue='Gender', data=train)
plt.title('Abalone age by gender', fontsize=30)
plt.xlabel('target(age)', fontsize=15)
plt.ylabel('Density', fontsize=15)
save_fig("Abalone age by gender")
plt.show()

In [ ]:
# 수치형 데이터 시각화
description = train.describe()
description

In [ ]:
# 수치형 데이터 통계치 그래프
interest_coloumns = train.columns[1:]
# plt.style.use('fivethirtyeight')
fig, ax =plt.subplots(2, 4, figsize = (25, 10))
fig.suptitle('Histogram of interesting features', fontsize=40)
column_idx = 0
for i in range(2):
    for j in range(4):
        ax[i][j].hist(train[interest_coloumns[column_idx]], bins=30, color='#eaa18a', edgecolor='#7bcabf')
        ax[i][j].set_title(interest_coloumns[column_idx])
        ax[i][j].axvline(description[interest_coloumns[column_idx]]['mean'], c='#f55354', label = f"mean = {round(description[interest_coloumns[column_idx]]['mean'], 2)}")
        ax[i][j].axvline(description[interest_coloumns[column_idx]]['50%'], c='#518d7d', label = f"median = {round(description[interest_coloumns[column_idx]]['50%'], 2)}")
        ax[i][j].legend()
        column_idx += 1
save_fig("Histogram of interesting features")

In [ ]:
def visualize(axx, yfield):
    sns.regplot(x='Target', y=yfield, data=train,  color='#eaa18a', line_kws=  {'color': '#f55354'} , ax = axx) # regplot을 이용하여 추세선과 산점도 그래프를 그려줍니다.
    axx.set_title(yfield)

In [ ]:
figure, ((ax1,ax2,ax3,ax4),(ax5,ax6,ax7,ax8)) = plt.subplots(nrows=2, ncols=4)
figure.set_size_inches(20,12)
figure.suptitle('Correlation between target and features', fontsize=40)

for i in range(len(train.columns[1:-1])):
    visualize(eval(f'ax{i+1}'), train.columns[i+1])
save_fig("Correlation between target and features")

In [ ]:
# 수치형 데이터 상관관계 히트맵
train_corr = train.drop(columns=['Gender'], axis=1)
corr = train_corr.corr(method='pearson')

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(data=corr, annot=True, fmt='.2f', linewidths=.5, cmap='Blues')
plt.title('Correlation between features', fontsize=30)
save_fig('Correlation between features')
plt.show()

In [ ]:
# target과 피처들의 상관관계
unstacked = corr.unstack()
unstacked_df = pd.DataFrame(unstacked['Target'].sort_values(ascending=False), columns=['Target'])
unstacked_df.style.background_gradient(cmap='viridis')

In [ ]:
train

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(x=train['Target'], y=train['Lenght'], hue=train['Gender'])
sns.boxplot(x=train['Target'], y=train['Diameter'], hue=train['Gender'])
sns.boxplot(x=train['Target'], y=train['Height'], hue=train['Gender'])
sns.boxplot(x=train['Target'], y=train['Whole Weight'], hue=train['Gender'])

## Modeling

In [12]:
le = LabelEncoder()
minmax = MinMaxScaler()

In [13]:
# labelencoder 
result = le.fit_transform(train['Gender'])
train_new = train.drop(['Gender', 'Target'], 1)
train_new['Gender_encoded'] = result

result = le.transform(test['Gender'])
test_new = test.drop(['id', 'Gender'], 1)
test_new['Gender_encoded'] = result

In [16]:
# minmaxscaler
scaled_train_new = minmax.fit_transform(train_new)
scaled_test_new = minmax.transform(test_new)

In [17]:
#pca
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
train_low = pca.fit_transform(scaled_train_new)
test_low = pca.transform(test_new)

In [18]:
X = train_low
y = train['Target']

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [19]:
# model 생성
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
mse_test

Epoch 1/20
22/22 [==============================] - 0s 4ms/step - loss: 102.6366 - val_loss: 92.4159
Epoch 2/20
22/22 [==============================] - 0s 958us/step - loss: 87.4436 - val_loss: 76.3967
Epoch 3/20
22/22 [==============================] - 0s 1ms/step - loss: 74.5189 - val_loss: 56.6870
Epoch 4/20
22/22 [==============================] - 0s 1ms/step - loss: 53.6235 - val_loss: 36.3995
Epoch 5/20
22/22 [==============================] - 0s 961us/step - loss: 33.2510 - val_loss: 20.8646
Epoch 6/20
22/22 [==============================] - 0s 912us/step - loss: 19.7874 - val_loss: 12.6828
Epoch 7/20
22/22 [==============================] - 0s 949us/step - loss: 12.8703 - val_loss: 9.5220
Epoch 8/20
22/22 [==============================] - 0s 934us/step - loss: 9.5284 - val_loss: 8.4699
Epoch 9/20
22/22 [==============================] - 0s 958us/step - loss: 9.3770 - val_loss: 8.0917
Epoch 10/20
22/22 [==============================] - 0s 895us/step - loss: 8.7185 - val_loss

6.228969097137451

In [ ]:
y_pred = model.predict(test_new).flatten()

In [ ]:
submission['Target'] = np.around(y_pred)
submission['Target'] = submission['Target'].astype('int')
submission.to_csv('data/submission.csv', index=False)

In [20]:
# model 생성
model2 = keras.models.Sequential([
    keras.layers.Dense(60, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model2.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
mse_test

Epoch 1/50
22/22 [==============================] - 0s 2ms/step - loss: 7.0243 - val_loss: 7.1665
Epoch 2/50
22/22 [==============================] - 0s 1ms/step - loss: 6.9804 - val_loss: 7.1449
Epoch 3/50
22/22 [==============================] - 0s 973us/step - loss: 6.9455 - val_loss: 7.1265
Epoch 4/50
22/22 [==============================] - 0s 885us/step - loss: 6.9197 - val_loss: 7.1102
Epoch 5/50
22/22 [==============================] - 0s 918us/step - loss: 6.8989 - val_loss: 7.0962
Epoch 6/50
22/22 [==============================] - 0s 893us/step - loss: 6.8742 - val_loss: 7.0843
Epoch 7/50
22/22 [==============================] - 0s 925us/step - loss: 6.8560 - val_loss: 7.0734
Epoch 8/50
22/22 [==============================] - 0s 912us/step - loss: 6.8373 - val_loss: 7.0639
Epoch 9/50
22/22 [==============================] - 0s 885us/step - loss: 6.8219 - val_loss: 7.0557
Epoch 10/50
22/22 [==============================] - 0s 921us/step - loss: 6.8081 - val_loss: 7.0478
Epo

5.788545608520508

In [22]:
y_pred = model.predict(test_low).flatten()

In [23]:
submission['Target'] = np.around(y_pred)
submission['Target'] = submission['Target'].astype('int')
submission.to_csv('data/submission.csv', index=False)

In [ ]:
# 단위가 다르기 때문에 스케일링 진행 -> minmaxscaler
# 우선 모델링한 다음에 ols 진행
# 그 다음에 피처 엔지니어링 
# polynomial
# pca 

In [ ]:
# 다중공산성

In [ ]:
train_new2 = train.drop(['Target'], 1)
train_new2 = pd.get_dummies(train_new2)

In [ ]:
test_new2 = test.drop(['id'], 1)
test_new2 = pd.get_dummies(test_new2)

In [ ]:
X = train_new2
y = train['Target']

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

X_train = minmax.fit_transform(X_train)
X_valid = minmax.transform(X_valid)
X_test = minmax.transform(X_test)

In [ ]:
test_new2 = minmax.transform(test_new2)

In [ ]:
# model 생성
model = keras.models.Sequential([
    keras.layers.Dense(60, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
mse_test

In [ ]:
y_pred = model.predict(test_new2).flatten()

In [ ]:
submission['Target'] = np.around(y_pred)
submission['Target'] = submission['Target'].astype('int')
submission.to_csv('data/submission.csv', index=False)

In [37]:
t

array([[-0.00704495,  0.49520471,  0.08459276,  0.13697928,  0.11770083],
       [-0.36364383, -0.18299772,  0.02321022,  0.04361766,  0.01045075],
       [ 0.3816532 ,  0.11420539, -0.03903539, -0.12084713, -0.00453558],
       ...,
       [ 0.13958769, -0.52059854,  0.06020322,  0.10127533, -0.01677944],
       [-0.10071185, -0.61635199,  0.00563003,  0.01808009, -0.01782461],
       [ 0.89627853, -0.21869038,  0.08302134,  0.0324764 , -0.04448994]])